## Calculation of $L_u$ and $M_r$
This notebook pages shows how to calculate $L_u$ and $M_r$ for an supported class 1 or 2 W shape by CSA S16-09 13.6 (a).

In [2]:
from math import pi, sqrt, ceil, log10
import inspect
from Designer import sst
from utils import sfround, call

In [3]:
Shp_W = 1
Shp_WWF = 5

In [4]:
SST = sst.SST()

In [5]:
def shape_class(B,T,D,W,A,Cf=0.,Fy=350.,phi=0.9):
    """Calculate the class of both flange and web and return
    both classes in a 2-tuple."""
    bovert = (B/2.0)/T
    hoverw = (D - 2.0*T)/W
    fklass = 4
    for i,L in [(1,145.0), (2,170.0), (3,200.0)]:
        if bovert <= L/sqrt(Fy):
            fklass = i
            break
    phiCy = phi*A*Fy*1E-3
    wklass = 4
    for i,L,S in [(1,1100.0,0.39),(2,1700.0,0.61), (3,1900.0,0.65)]:
        if hoverw <= (L/sqrt(Fy))*(1.0-S*Cf/phiCy):
            wklass = i
            break

    return (fklass,wklass)

In [6]:
call(shape_class,SST.section('WWF2000x732'),Fy=350.)

(1, 3)

In [7]:
def calcLu(B,T,D,W,A,Ix,Sx,Zx,Iy,J,Cw,E=200000.,G=77000.,Cf=0.,Fy=350.,phi=0.9):
    
    """Calculate Lu - max unsupported length of compression flange for
    which there is no reduction in strength below phi Mp (or phi My
    for class 3). For w2 = 1"""

    fclass,wclass = call(shape_class,locals())
    sclass = max(fclass,wclass)
    if wclass > 3:
        return 0.0

    Se = Sx
    if fclass == 4 and wclass < 4:
        bop = (B/2.0) - 200.0*T/sqrt(Fy)
        Se = (Ix - 4.0*bop*T*(D/2.0 - T/2.0)**2)/(D/2.0)

    if sclass in [1,2]:
        Mlim = Zx * Fy
    else:
        Mlim = Se * Fy

    P = E*Iy*G*J
    Q = Iy*Cw*(pi*E)**2
    w2 = 1.0

    # the following was generated by Maxima
    Lu = 0.06741626595263361*w2**0.5*((103684.0*Mlim**2*Q+55516.52475612764*w2**2*P**2)**0.5 + 235.61944901923448*w2*P)**0.5/Mlim

    return Lu

In [8]:
sfround(call(calcLu,SST.section('WWF1400x358'),Fy=350.))

5000.0

Compare with 5000 in the Beam Selection Tables.

In [9]:
def calcLuMr(Shp,B,T,D,W,A,Ix,Sx,Zx,Iy,J,Cw,E=200000.,G=77000.,Cf=0.,Fy=350.,phi=0.9,omega2=1.0,L=0.):

    """Calculate both Lu and Mr for the given shape with unsupported 
    compression flange.  Return both in a tuple."""
    
    if Shp not in [Shp_W,Shp_WWF]:
        raise Exception, "Only W and WWF shapes are implemented."

    fclass,wclass = call(shape_class,locals())
    sclass = max(fclass,wclass)
    ##if sclass == 3:
    ##    add_note( 'Class 3 section.' )

    Se = Sx
    if fclass == 4:
        if wclass < 4:
            bop = (B/2.0) - 200.0*T/sqrt(Fy)
            Se = (Ix - 4.0*bop*T*(D/2.0 - T/2.0)**2)/(D/2.0)
            #add_note( 'Class 4 flange: Se = %.3g by Clause 13.5(c)(iii).' % (Se,) )
        else:
            raise Exception, "Flange and web are both class 4."

    Mlim = Fy * (Zx if sclass in [1,2] else Se)
    P = E*Iy*G*J
    Q = Iy*Cw*(pi*E)**2
    # the following was generated by Maxima:
    w2 = omega2
    Lu = 0.067416*w2**0.5*((103684.0*Mlim**2*Q+55516.5*w2**2*P**2)**0.5 + 235.619*w2*P)**0.5/Mlim
    Lu = Lu

    if L == 0.0:
        if sclass in [1,2]:
            Mp = Zx * Fy * 1.0E-6
            Mr = phi * Mp
        elif sclass in [3] or (fclass == 4 and wclass <= 3):
            My = Se * Fy * 1.0E-6
            Mr = phi * My
        else:
            #add_note( 'Class 4 webs not implemented.' )
            Mr =  0.0
    else:
        Mu = (omega2*pi/L) * sqrt(E*Iy*G*J + Iy*Cw*(pi*E/L)**2 ) *1.0E-6

        if sclass in [1,2]:
            Mp = Fy * Zx * 1.0E-6
            if Mu > 0.67*Mp:
                Mr = 1.15*phi*Mp*(1.0 - 0.28*Mp/Mu)
                Mr = min( Mr, phi*Mp )
            else:
                Mr = phi * Mu
        elif sclass in [3] or (fclass == 4 and wclass <= 3):
            My = Fy * Se * 1.0E-6
            if Mu > 0.67*My:
                Mr = 1.15*phi*My*(1.0 - 0.28*My/Mu)
                Mr = min( Mr, phi*My )
            else:
                Mr = phi * Mu
        else:
            add_note( 'Class 4 webs not implemented.' )
            Mr = 0.0

    return Lu,Mr

In [10]:
[sfround(x) for x in call(calcLuMr,SST.section('WWF1800x510'),Fy=350.,L=20000.)]

[5960.0, 2940.0]

Compare with 5960 and 2940 in the Beam Slection Tables.